In [1]:
import json
import random
import sys
import os

# Add the project root directory to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '../../..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

random.seed(2)

from src.agents.context_qa_correction_agent import ContextQACorrectionAgent

In [2]:
with open(f"../../../dataset/raw_model_responses/train/train_rajpurkar_squad.json", "r") as f:
    json_data = json.load(f)

In [3]:
json_data[0]

{'task_info': {'type': 'Contextual QA', 'dataset': 'rajpurkar_squad'},
 'additional_info': {'model': 'meta-llama/Llama-3.2-1B-Instruct',
  'question': 'What percentage of Egyptians polled support death penalty for those leaving Islam?',
  'context': 'The Pew Forum on Religion & Public Life ranks Egypt as the fifth worst country in the world for religious freedom. The United States Commission on International Religious Freedom, a bipartisan independent agency of the US government, has placed Egypt on its watch list of countries that require close monitoring due to the nature and extent of violations of religious freedom engaged in or tolerated by the government. According to a 2010 Pew Global Attitudes survey, 84% of Egyptians polled supported the death penalty for those who leave Islam; 77% supported whippings and cutting off of hands for theft and robbery; and 82% support stoning a person who commits adultery.',
  'answer': ['84%'],
  'title': 'Egypt'},
 'input': "<|begin_of_text|><|s

In [9]:
index = 61
# index = 0
responses = json_data[index]["response"]

question = json_data[index]["additional_info"]["question"]
context = json_data[index]["additional_info"]["context"]
answer = json_data[index]["additional_info"]["answer"]
print(question)
print(answer)

Why are older neighborhoods in San Diego popular with cyclists?
['the grid street patterns']


In [10]:
responses

['Older and denser neighborhoods around downtown San Diego are popular with cyclists because they are more suitable for cycling due to their grid street patterns, which are less conducive to car traffic.',
 'Older and denser neighborhoods around the downtown tend to be utility cycling oriented because of the grid street patterns that are absent in newer developments farther from the urban core, where suburban style arterial roads are more common.',
 'Older and denser neighborhoods around the downtown tend to be more suitable for cycling due to the grid street patterns absent in newer developments farther from the urban core, which result in more direct access to amenities and services, making cycling a convenient option for these areas.',
 'Older neighborhoods around the downtown tend to be utility cycling oriented because of the grid street patterns absent in newer developments farther from the urban core, which are more conducive to suburban-style arterial roads.',
 'Older neighborho

In [11]:
agent = ContextQACorrectionAgent(error_detection_only=False)

state = await agent.model.ainvoke({
    "question": question,
    "context": context,
    "answer": answer,
    "responses": responses,
})

You are an expert Error Detection AI. Your primary goal is to identify factual, semantic, and grammatical inaccuracies in a given `Response to Evaluate` by comparing it against a `Context` and a `Correct Answer`.

# Core Principles
1. **Focus on Substance**: Your analysis must target factual inaccuracies, semantic deviations from the `Correct Answer`, logical contradictions, and grammatical errors.
2. **Tolerate Benign Additions**: Do NOT flag stylistic choices, verbosity, or extra information that is factually correct (supported by the `Context`) and does not undermine the correctness of the core answer. For example, introductory phrases like 'According to the context...' are not errors.
3. **Be Precise and Objective**: Every identified error must be grounded in the provided materials. Your output must be a structured list, with no conversational filler.

# Inputs
**Question**: Why are older neighborhoods in San Diego popular with cyclists?
**Context**: '''San Diego's roadway system p

2025-08-02 19:13:01.501 | INFO     | src.agents.context_qa_correction_agent:check_for_errors:102 - Errors: [[Error(description="The response incorrectly states that grid street patterns are 'less conducive to car traffic'. The context does not support this claim.", location='which are less conducive to car traffic', correction='DELETE: which are less conducive to car traffic')], [], [], [], [], [], [], [], [Error(description='The response provides additional information not found in the context or correct answer. The question asks specifically about why older neighborhoods are popular with cyclists, and the context only mentions the grid street patterns.', location='Older neighborhoods in San Diego are popular with cyclists because they are denser and have grid street patterns, making it easier to navigate and access amenities, whereas newer developments with suburban-style arterial roads are farther from the urban core.', correction='DELETE: Older neighborhoods in San Diego are popula

You are a Master Editor AI. Your mission is to correct a flawed response with surgical precision using a special tag-based editing language.

# Core Principles:
1. **Precision is Paramount**: Your edits must be exact. Every tag you use has a specific function. Use them flawlessly.
2. **The Principle of Efficiency**: Your goal is to be efficient. Use `<DEL_W>` for small edits and `<DEL_S>` or `<DEL_A>` for larger ones. **Rule**: You may use up to **three** `<DEL_W>` tags within a single sentence. If a sentence requires deleting more than three words, you MUST use `<DEL_S>` or `<DEL_A>` instead.
3. **Grammatical Perfection**: The final, corrected response must be grammatically flawless and semantically identical to the `Correct Answer`.

# Tag-Based Editing Language:
Your edits are performed by inserting tags and new text into the `Incorrect Response`. You have three deletion tags. If a correction only requires adding text, no tag is needed.

# How to use <DEL_W>: The Word Deleter
- **Fu

2025-08-02 19:13:02.934 | INFO     | src.agents.context_qa_correction_agent:correct_response:174 - Corrected responses: ['Older and denser neighborhoods around downtown San Diego are popular with cyclists because they are more suitable for cycling due to their grid street patterns, which are less conducive to car traffic<DEL_W>.', 'Older neighborhoods in San Diego are popular with cyclists because they are denser and have grid street patterns, making it easier to navigate and access amenities, whereas newer developments with suburban-style arterial roads are farther from the urban core.<DEL_A> the grid street patterns']


You are a meticulous AI Verification Engine. Your sole purpose is to determine if a `Generated Answer` is correct by strictly comparing it against a `Correct Answer` and its supporting `Context`.

# Task Inputs
**Question**: Why are older neighborhoods in San Diego popular with cyclists?
**Context**: '''San Diego's roadway system provides an extensive network of routes for travel by bicycle. The dry and mild climate of San Diego makes cycling a convenient and pleasant year-round option. At the same time, the city's hilly, canyon-like terrain and significantly long average trip distances—brought about by strict low-density zoning laws—somewhat restrict cycling for utilitarian purposes. Older and denser neighborhoods around the downtown tend to be utility cycling oriented. This is partly because of the grid street patterns now absent in newer developments farther from the urban core, where suburban style arterial roads are much more common. As a result, a vast majority of cycling-related

2025-08-02 19:13:03.418 | INFO     | src.agents.context_qa_correction_agent:verify_corrected_response:207 - Verified responses: ['True', 'True']


In [12]:
state

{'question': 'Why are older neighborhoods in San Diego popular with cyclists?',
 'context': "San Diego's roadway system provides an extensive network of routes for travel by bicycle. The dry and mild climate of San Diego makes cycling a convenient and pleasant year-round option. At the same time, the city's hilly, canyon-like terrain and significantly long average trip distances—brought about by strict low-density zoning laws—somewhat restrict cycling for utilitarian purposes. Older and denser neighborhoods around the downtown tend to be utility cycling oriented. This is partly because of the grid street patterns now absent in newer developments farther from the urban core, where suburban style arterial roads are much more common. As a result, a vast majority of cycling-related activities are recreational. Testament to San Diego's cycling efforts, in 2006, San Diego was rated as the best city for cycling for U.S. cities with a population over 1 million.",
 'answer': ['the grid street p